## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
movies


,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


## Data Preprocessing

In [3]:
training_set = pd.read_csv('training_set.csv', delimiter=',',on_bad_lines='skip' , header=None)
# Remove or skip the header row if it exists
training_set = training_set.iloc[1:]

# Fill NaN values with a suitable integer (e.g., 0)
training_set = training_set.fillna(0)

# Now convert to NumPy array
training_set = np.array(training_set, dtype='int')
total_elements = training_set.size
print(total_elements)

test_set = pd.read_csv('test_set.csv', delimiter=',', on_bad_lines='skip', header=None)
# Remove or skip the header row if it exists
test_set = test_set.iloc[1:]

# Fill NaN values with a suitable integer (e.g., 0)
test_set = test_set.fillna(0)

# Now convert to NumPy array
test_set = np.array(training_set, dtype='int')


max(test_set[:, 1])
max(training_set[:, 1])

nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
print(nb_users)
print(nb_movies)

<ipython-input-3-e8d04822504b>:1: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  training_set = pd.read_csv('training_set.csv', delimiter=',',on_bad_lines='skip' , header=None)


3000484


<ipython-input-3-e8d04822504b>:13: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  test_set = pd.read_csv('test_set.csv', delimiter=',', on_bad_lines='skip', header=None)


6040
3952


In [4]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)
test_set

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.]])

### build auto **encoder**

In [5]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the  model on the dataset

In [6]:
nb_epoch = 250
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.3479)
epoch: 2loss: tensor(1.0102)
epoch: 3loss: tensor(0.9899)
epoch: 4loss: tensor(0.9833)
epoch: 5loss: tensor(0.9802)
epoch: 6loss: tensor(0.9785)
epoch: 7loss: tensor(0.9774)
epoch: 8loss: tensor(0.9765)
epoch: 9loss: tensor(0.9759)
epoch: 10loss: tensor(0.9756)
epoch: 11loss: tensor(0.9753)
epoch: 12loss: tensor(0.9751)
epoch: 13loss: tensor(0.9748)
epoch: 14loss: tensor(0.9746)
epoch: 15loss: tensor(0.9744)
epoch: 16loss: tensor(0.9743)
epoch: 17loss: tensor(0.9741)
epoch: 18loss: tensor(0.9740)
epoch: 19loss: tensor(0.9741)
epoch: 20loss: tensor(0.9738)
epoch: 21loss: tensor(0.9739)
epoch: 22loss: tensor(0.9738)
epoch: 23loss: tensor(0.9736)
epoch: 24loss: tensor(0.9732)
epoch: 25loss: tensor(0.9726)
epoch: 26loss: tensor(0.9720)
epoch: 27loss: tensor(0.9712)
epoch: 28loss: tensor(0.9706)
epoch: 29loss: tensor(0.9698)
epoch: 30loss: tensor(0.9691)
epoch: 31loss: tensor(0.9686)
epoch: 32loss: tensor(0.9670)
epoch: 33loss: tensor(0.9662)
epoch: 34loss: tens

## testing the results

In [8]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

id_user = 6  # Example: Predicting for test user 10
id_movie = 17  # Example: Predicting for test movie 8

predicted_rating = output[0, id_movie - 1].item()  # id_movie - 1 due to zero indexing

# Print the predicted rating
print(f"Predicted rating for test user {id_user} on test movie {id_movie}: {predicted_rating}")

test loss: tensor(0.8731)
Predicted rating for test user 10 on test movie 8: 0.0
